In [75]:
def get_data(filename):
    with open(filename) as f:
        return int(f.readline(), 16)


def get_segment(value: int, mask: int, offset=0) -> int:
    return value >> (value.bit_length() - offset) & mask


def get_packet_metadata(value, offset=0):
    ver = get_segment(value, 0b111, 3 + offset)
    typ = get_segment(value, 0b111, 6 + offset)
    return ver, typ


def read_packets(value: int, offset=0):
    """
    the offset must be set at 6 to exclude ver and type of packet
    """
    currentLocation = offset + 6
    packets = []
    hasNext = 1
    while hasNext:
        currentLocation += 1
        hasNext = get_segment(value, 0b1, currentLocation)

        currentLocation += 4
        pkg = get_segment(value, 0b1111, currentLocation)
        packets.append(pkg)

        if currentLocation > value.bit_length() + 5:
            raise Exception("failed...")

    return packets, currentLocation


def literal_value(packets):
    return int("".join([f"{p:04b}" for p in packets]), 2)


def get_L_value(val):
    i_val = get_segment(val, 0b1, 7)
    l_mask_len = (0x7FF, 11 + 7) if i_val else (0x7FFF, 15 + 7)
    return i_val, get_segment(val, *l_mask_len), l_mask_len[1]


def get_sub_packet(val, start, end):
    length = end - start
    mask = (1 << length) - 1
    return get_segment(val, mask, end)


def correction_bit(val: int):
    if val.bit_length() % 4 == 0:
        return 0
    return 1 << ((val.bit_length() % 4) + (val.bit_length() - 1))

def find_literal_offset(value:int) -> int:
    offset = 0
    _, typ = get_packet_metadata(value, offset)
    while (typ != 4):
        if offset < -4: return 0 
        offset -= 1
        _, typ = get_packet_metadata(value, offset)
    return offset


def handel_sub_packets(subpkg, i_val: int, l_data: int):
    result = []
    offset = find_literal_offset(subpkg)
    if i_val == 0:
        while offset < l_data:
            v, t = get_packet_metadata(subpkg, offset)
            p, offset = read_packets(subpkg, offset)
            result.append((v, t, literal_value(p)))
    elif i_val == 1:
        count = 0
        while count < l_data:
            v, t = get_packet_metadata(subpkg, offset)
            p, offset = read_packets(subpkg, offset)
            result.append((v, t, literal_value(p)))
            count += 1
    else:
        raise Exception

    return result


### Sample data 1

In [2]:
val = 0XD2FE28
packets,offset = read_packets(val)
print(packets,offset)
literal_value(packets)

[7, 14, 5] 21


2021

### Sample data 2

In [76]:
# data = get_data("input")
data = 0x38006F45291200
correction = correction_bit(data)
print(data.bit_length() % 4)
# print(bin(data))
data |= correction
ver, typ = get_packet_metadata(data)
# print(bin(correction))
# print(bin(data))
print(ver, typ)
if correction > 0 :
    print("test", data.bit_length() % 4)
    ver &= 0b011
print(f"{ver= }, {typ= }")
i, l, new_offset = get_L_value(data)
# new_offset -= 1
print(i,l, new_offset)
print(new_offset, new_offset+l, data.bit_length())
# get sub pkg
subpkg = get_sub_packet(data, new_offset, data.bit_length())
print(bin(subpkg))
handel_sub_packets(subpkg, i, l)

2
5 6
test 0
ver= 1, typ= 6
0 27 22
22 49 56
0b1101000101001010010001001000000000


[(6, 4, 10), (2, 4, 20)]

In [48]:
0b1101000101001010010001001000000000.bit_length()%4

2

### Sample Data 3

In [80]:
# data = get_data("input")
data = 0xEE00D40C823060
correction = correction_bit(data)
print(data.bit_length() % 4)
# print(bin(data))
data |= correction
ver, typ = get_packet_metadata(data)
# print(bin(correction))
# print(bin(data))
print(ver, typ)
if correction > 0 :
    print("test", data.bit_length() % 4)
    ver &= 0b011
print(f"{ver= }, {typ= }")
i, l, new_offset = get_L_value(data)
# new_offset -= 1
print(i,l, new_offset)
print(new_offset, new_offset+l, data.bit_length())
# get sub pkg
subpkg = get_sub_packet(data, new_offset, data.bit_length())
print(bin(subpkg))
handel_sub_packets(subpkg, i, l)

0
7 3
ver= 7, typ= 3
1 3 18
18 21 56
0b1010000001100100000100011000001100000


[(2, 4, 1), (4, 4, 2), (1, 4, 3)]

In [93]:
def handel_packets(value):
    correction = correction_bit(value)
    value |= correction
    ver, typ = get_packet_metadata(value)
    if correction > 0 :
        print("test", data.bit_length() % 4)
        ver &= 0b011


In [92]:
0b1010000001100100000100011000001100000.bit_length() % 4

1

In [74]:
find_literal_offset(0b1000001100100000100011000001100000)

-3

In [9]:
p, _ = read_packets(subpkg, offset=11)
literal_value(p)

20

In [95]:
bin(0x8A004A801A8002F478)

0

In [91]:
print(bin(0xa2), )
print(bin(0xff), )
print(bin(0x62 & 0xff))
print(bin(0xf << 4))

0b10100010
0b11111111
0b1100010
0b11110000
